Introduction to using OCP in molecular simulations
==================================================

# Calculating adsorption energies

We begin with a classical task of computing an adsorption energy in a classical way. 

Adsorption energies are different than you might be used to in OCP. For example, you may want the adsorption energy of oxygen, and conventionally you would compute that from this reaction:

    1/2 O2 + slab -> slab-O

We can use OCP to get the total energy of the `slab` and `slab-O`, but we will need another way to get the energy of `1/2 O2`

First we need to make sure we have the checkpoint we want. This is a structure to energy and forces task (S2EF).

In [1]:
import os
import requests

checkpoint = 'escn_l6_m3_lay20_all_md_s2ef.pt'

if not os.path.exists(checkpoint):
    url = 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m3_lay20_all_md_s2ef.pt'
    #url = 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_oc20_all_s2ef.pt'
    with open(checkpoint, 'wb') as f:
        f.write(requests.get(url).content)

Now, we can't compute adsorption energies directly; OCP can give us the energy of slabs and slabs + adsorbates, but it is not trained to give us molecular energies. We need to get these from another source. The ideal source is a DFT calculation that is done consistently with the data that your OCP checkpoint was trained from. An alternative that is less ideal is to use an experimental reference energy. This is considered less ideal because there are known systematic errors between DFT functionals and experimental reference energies, and you will have to do some work to apply a correction for this. 



In [2]:
%%capture
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
calc = OCPCalculator(checkpoint=os.path.expanduser(checkpoint))

EOFError: Ran out of input

In [12]:
import numpy as np
from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms, FixScaled
from ase.optimize import BFGS

re1 = -3.03 # Water formation energy in eV

In [3]:
slab = fcc111('Pt', size=(2, 2, 3), vacuum=10.0)
add_adsorbate(slab, 'O', height=1.2, position='fcc')

slab.set_calculator(calc)
opt = BFGS(atoms)
opt.run(fmax=0.05, steps=100)
slab_e = slab.get_potential_energy()
slab_e

NameError: name 'fcc111' is not defined

In [16]:
add_adsorbate(slab, 'O', height=1.2, position='fcc')

opt = BFGS(slab)

opt.run(fmax=0.05, steps=100)
ads_slab_e = atoms.get_potential_energy()
ads_slab_e

      Step     Time          Energy         fmax
BFGS:    0 16:03:38      -67.119247        1.6295
BFGS:    1 16:03:39      -67.162376        0.9006
BFGS:    2 16:03:39      -67.237015        0.6303
BFGS:    3 16:03:40      -67.254639        0.6923
BFGS:    4 16:03:40      -67.348862        0.5711
BFGS:    5 16:03:41      -67.434280        0.2473
BFGS:    6 16:03:41      -67.424904        0.1869
BFGS:    7 16:03:42      -67.435570        0.1563
BFGS:    8 16:03:42      -67.465027        0.0894
BFGS:    9 16:03:43      -67.473831        0.0520
BFGS:   10 16:03:43      -67.482109        0.0264


-67.20462036132812